**--- Inicio Colab ---**


In [1]:
# Monto drive en colab
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
!cp '/content/gdrive/My Drive/Colab Notebooks/tp2-datos/feature_builder.py' '/content'
!cp '/content/gdrive/My Drive/Colab Notebooks/tp2-datos/util.py' '/content'
!cp '/content/gdrive/My Drive/Colab Notebooks/tp2-datos/hyperparameter_tuning.py' '/content'
!cp '/content/gdrive/My Drive/Colab Notebooks/tp2-datos/feature_builder.py' '/content'
!cp '/content/gdrive/My Drive/Colab Notebooks/tp2-datos/train.csv' '/content'
!cp '/content/gdrive/My Drive/Colab Notebooks/tp2-datos/test.csv' '/content'
!mkdir /content/data/embeddings
!wget -P /content/data/embeddings -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"
!gunzip '/content/data/embeddings/GoogleNews-vectors-negative300.bin.gz'
!mv '/content/data/embeddings/GoogleNews-vectors-negative300.bin' '/content/data/embeddings/word2vec.bin'

mkdir: cannot create directory ‘/content/data/embeddings’: File exists
--2020-07-23 04:09:28--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.186.205
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.186.205|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1647046227 (1.5G) [application/x-gzip]
Saving to: ‘/content/data/embeddings/GoogleNews-vectors-negative300.bin.gz’

GoogleNews-vectors- 100%[===================>]   1.53G  74.8MB/s    in 25s     

2020-07-23 04:09:53 (62.3 MB/s) - ‘/content/data/embeddings/GoogleNews-vectors-negative300.bin.gz’ saved [1647046227/1647046227]



In [3]:
!ls -l '/content/data/embeddings/word2vec.bin'

-rw-r--r-- 1 root root 3644258522 Mar  5  2015 /content/data/embeddings/word2vec.bin


In [6]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

**--- Fin Colab ---**

In [19]:
import pandas as pd
import numpy as np
import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing import sequence
from tensorflow import keras
from tensorflow.keras import layers
from gensim.models import KeyedVectors
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Dense, Conv1D, Flatten, MaxPooling1D, Embedding, GlobalMaxPooling1D, Dropout, LSTM,Input,Activation,Bidirectional,GRU
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import nltk
from sklearn.metrics import f1_score
import feature_builder
import util
import hyperparameter_tuning
embeddings = None
from hyperopt import hp,STATUS_OK

In [7]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
Y = train_df['target'].values

In [8]:
def prepare_for_cnn(df):
    processed = feature_builder.process_dataset(df)
    return (processed, processed.to_numpy().reshape(processed.shape[0], 1, processed.shape[1]))

In [9]:
processed, X_processed = prepare_for_cnn(train_df)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


Embeddings loaded!
Percentage of words covered in the embeddings = 0.4937444933920705
Embeddings loaded!
Percentage of words covered in the embeddings = 0.5879980378263476


In [10]:
X_train, X_test, y_train, y_test = train_test_split(X_processed,Y,test_size = 0.33,random_state = 17)

In [22]:
def RNN():
    model = tf.keras.Sequential([
    tf.keras.layers.Bidirectional(tf.keras.layers.GRU(64),input_shape=(X_train.shape[1], X_train.shape[2])),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1)])
    return model

In [24]:
model = RNN()
model.summary()
model.compile(loss=keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_5 (Bidirection (None, 128)               271104    
_________________________________________________________________
dense_10 (Dense)             (None, 64)                8256      
_________________________________________________________________
dropout_5 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 65        
Total params: 279,425
Trainable params: 279,425
Non-trainable params: 0
_________________________________________________________________


In [25]:
history = model.fit(X_train, y_train, epochs=50, shuffle=False, validation_split=0.0)
util.plot_history(history)

Epoch 1/50
160/160 [==============================] - 1s 6ms/step - loss: 0.7171 - accuracy: 0.5692
Epoch 2/50
160/160 [==============================] - 1s 6ms/step - loss: 0.6518 - accuracy: 0.5888
Epoch 3/50
160/160 [==============================] - 1s 6ms/step - loss: 0.6246 - accuracy: 0.6127
Epoch 4/50
160/160 [==============================] - 1s 5ms/step - loss: 0.6045 - accuracy: 0.6288
Epoch 5/50
160/160 [==============================] - 1s 5ms/step - loss: 0.5797 - accuracy: 0.6547
Epoch 6/50
160/160 [==============================] - 1s 6ms/step - loss: 0.5659 - accuracy: 0.6839
Epoch 7/50
160/160 [==============================] - 1s 5ms/step - loss: 0.5488 - accuracy: 0.7035
Epoch 8/50
160/160 [==============================] - 1s 5ms/step - loss: 0.5298 - accuracy: 0.7229
Epoch 9/50
160/160 [==============================] - 1s 5ms/step - loss: 0.5187 - accuracy: 0.7333
Epoch 10/50
160/160 [==============================] - 1s 5ms/step - loss: 0.4984 - accuracy: 0.7449

KeyError: ignored

In [26]:
prediction = model.predict_classes(X_test)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [27]:
accuracy_score(y_test, prediction)

0.8018304814962196

In [28]:
f1_score(y_test, prediction)

0.7275711159737417

Hyperopt (bayesian)

In [29]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
Y = train_df['target'].values

In [30]:
processed = feature_builder.process_dataset(train_df)
X_processed = processed.to_numpy().reshape(processed.shape[0], 1, processed.shape[1])
X_train, X_test, y_train, y_test = train_test_split(X_processed,Y,test_size = 0.33,random_state = 17)

Embeddings loaded!
Percentage of words covered in the embeddings = 0.4937444933920705
Embeddings loaded!
Percentage of words covered in the embeddings = 0.5879980378263476


In [40]:
def gru(params):
    model = Sequential()
    model.add(Bidirectional(GRU(params['units']),input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(Dense(params['units2'], activation='relu'))
    model.add(Dropout(params['dropout']))
    model.add(Dense(1))
    
    
    model.compile(loss=keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])
    
    result = model.fit(X_train, y_train, epochs=50, shuffle=True, validation_split=0.2,verbose=0)
    prediction = model.predict_classes(X_test)
    score = f1_score(y_test, prediction)
    #get the highest validation accuracy of the training epochs
    #validation_acc = np.amax(result.history['val_accuracy']) 
    #print('Best validation acc of epoch:', validation_acc)
    print('\nScore:',score)
    return {'loss': -score, 'status': STATUS_OK, 'model': model}


In [41]:
space = {'units': hp.choice('units',[16,32,64,128]),
        'units2': hp.choice('units2',[16,32,64,128]),
        'dropout': hp.choice('dropout',[0.1,0.2,0.5,0.7,0.9])}

In [42]:
hyperparameter_tuning.bayesian_optimization(gru,space,max_evals=100)


Score:
0.7184035476718404

Score:
0.7384615384615385

Score:
0.19966722129783696

Score:
0.7298474945533768

Score:
0.7636549259826442

Score:
0.7489583333333333

Score:
0.7435492364402315

Score:
0.7084499160604365

Score:
0.7292576419213974

Score:
0.7530279094260136

Score:
0.7514095335725269

Score:
0.7166756610901241

Score:
0.7645320197044335

Score:
0.7674876847290639

Score:
0.7623363544813696

Score:
0.7708333333333334

Score:
0.7535934291581108

Score:
0.7092829349638688

Score:
0.749090909090909

Score:
0.7366167023554604

Score:
0.736209721463681

Score:
0.7507853403141361

Score:
0.7652799178222908

Score:
0.671272308578008

Score:
0.7510373443983401

Score:
0.7657657657657658

Score:
0.7467087941021592

Score:
0.7477525118984665

Score:
0.7498689040377556

Score:
0.7244953627932351

Score:
0.7651821862348178

Score:
0.7375661375661376

Score:
0.7390396659707724

Score:
0.686753837407618

Score:
0.7392

Score:
0.7583791895947974

Score:
0.7482993197278911

Score:
0.725542